# Python OOP (Object-Oriented Programming) — Detailed Theory + Examples

OOP helps you model real-world and software concepts using **objects** that bundle **data (state)** and **behavior (methods)**.

In ML projects, OOP is commonly used to build:
- Data loaders / preprocessors
- Model wrappers (training, evaluation, saving/loading)
- Pipelines and reusable components
- Metrics, callbacks, configuration objects

## How to use this notebook
- Run cells top-to-bottom.
- Code cells contain **numbered examples** (Example 1, Example 2, …).
- Read the theory before executing the examples in that section.


## 1) Core concepts: Class, Object, Attributes, Methods

### What is a class?
A **class** is a blueprint that defines:
- **Attributes** (data): variables stored on an object
- **Methods** (behavior): functions that operate on the object’s data

### What is an object?
An **object** is an instance created from a class. Each object has its own state.

### Instance attributes vs class attributes
- **Instance attribute**: stored per-object (each instance can differ)
- **Class attribute**: stored on the class (shared by all instances unless overridden)

### `self`
In instance methods, the first parameter is traditionally named `self`.
- It refers to the current instance.
- Python automatically passes it when you call `obj.method(...)`.

### `__init__`
`__init__` is the initializer.
- It runs right after object creation.
- You typically set up instance attributes here.


In [ ]:
# Examples 1–5: class, object, attributes, methods

# Example 1: simplest class
class Dog:
    pass

d = Dog()
print("Example 1:", d, type(d))

# Example 2: __init__ + instance attributes
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

p = Point(3, 4)
print("Example 2:", p.x, p.y)

# Example 3: instance method uses self
class Greeter:
    def __init__(self, name):
        self.name = name

    def greet(self):
        return f"Hello, {self.name}!"

g = Greeter("Ada")
print("Example 3:", g.greet())

# Example 4: class attribute shared by default
class Counter:
    created = 0  # class attribute

    def __init__(self):
        Counter.created += 1

c1 = Counter()
c2 = Counter()
print("Example 4:", "created=", Counter.created)

# Example 5: instance attribute overrides class attribute for that instance
c1.created = 999  # now an instance attribute on c1
print("Example 5:", "c1.created=", c1.created, "Counter.created=", Counter.created)


## 2) Representation: `__repr__` and `__str__`

### Why these matter
Readable objects make debugging and logging easier.
- `__repr__`: developer-facing representation (aim for unambiguous)
- `__str__`: user-facing representation (aim for readability)

When you print an object:
- `print(obj)` prefers `__str__` if available, otherwise falls back to `__repr__`.
- In notebooks, the last expression often displays `__repr__`.


In [ ]:
# Examples 6–7: __repr__ and __str__

# Example 6: __repr__
class Vector2D:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __repr__(self):
        return f"Vector2D(x={self.x}, y={self.y})"

v = Vector2D(1.5, -2)
print("Example 6:", v)

# Example 7: __str__ vs __repr__
class User:
    def __init__(self, username):
        self.username = username

    def __repr__(self):
        return f"User(username={self.username!r})"

    def __str__(self):
        return self.username

u = User("suvom")
print("Example 7 print:", u)
print("Example 7 repr:", repr(u))


## 3) Encapsulation: controlling access with conventions + `property`

### Encapsulation in Python
Python does not enforce access modifiers like some languages (e.g., `private`, `protected`) as strict rules. Instead it uses **conventions**:
- `name`: public
- `_name`: “internal use” (soft private)
- `__name`: name-mangled (harder to access accidentally)

### Why use `property`?
A `property` lets you expose an attribute-like interface while still validating or computing values.

Common use-cases:
- Validate inputs (e.g., non-negative age)
- Keep backward compatibility while changing internal storage
- Create computed fields (e.g., `area` based on `width` and `height`)


In [ ]:
# Examples 8–10: encapsulation + property

# Example 8: underscore convention
class BankAccount:
    def __init__(self, balance=0):
        self._balance = float(balance)  # internal attribute

    def deposit(self, amount):
        if amount <= 0:
            raise ValueError("amount must be positive")
        self._balance += amount

    def withdraw(self, amount):
        if amount <= 0:
            raise ValueError("amount must be positive")
        if amount > self._balance:
            raise ValueError("insufficient funds")
        self._balance -= amount

    def get_balance(self):
        return self._balance

acct = BankAccount(100)
acct.deposit(25)
acct.withdraw(30)
print("Example 8:", acct.get_balance())

# Example 9: property for controlled read access
class Temperature:
    def __init__(self, celsius):
        self._c = float(celsius)

    @property
    def celsius(self):
        return self._c

    @property
    def fahrenheit(self):
        return self._c * 9 / 5 + 32

t = Temperature(25)
print("Example 9:", t.celsius, t.fahrenheit)

# Example 10: property with validation on set
class Person:
    def __init__(self, age):
        self.age = age  # triggers setter

    @property
    def age(self):
        return self._age

    @age.setter
    def age(self, value):
        value = int(value)
        if value < 0:
            raise ValueError("age cannot be negative")
        self._age = value

p = Person(30)
print("Example 10:", p.age)


## 4) Class methods and static methods

### Regular (instance) methods
- First parameter is `self`.
- Operate on instance state.

### `@classmethod`
- First parameter is `cls` (the class).
- Often used as **alternative constructors** (create objects from different input formats).

### `@staticmethod`
- No automatic `self` or `cls`.
- Used for helper logic that conceptually belongs to the class’s namespace.


In [ ]:
# Examples 11–12: classmethod and staticmethod

# Example 11: classmethod as alternative constructor
class Date:
    def __init__(self, year, month, day):
        self.year = int(year)
        self.month = int(month)
        self.day = int(day)

    @classmethod
    def from_iso(cls, iso_string):
        y, m, d = iso_string.split("-")
        return cls(y, m, d)

    def __repr__(self):
        return f"Date({self.year}, {self.month}, {self.day})"

d = Date.from_iso("2026-01-17")
print("Example 11:", d)

# Example 12: staticmethod helper
class Math:
    @staticmethod
    def clamp(x, low, high):
        return max(low, min(high, x))

print("Example 12:", Math.clamp(10, 0, 5), Math.clamp(-3, 0, 5))


## 5) Inheritance and `super()`

### Inheritance
Inheritance lets a class (child/subclass) reuse and extend behavior from another class (parent/superclass).

Use inheritance when there is a clear **is-a** relationship:
- A `Cat` **is a** `Animal`
- A `LinearRegressionModel` **is a** `Model`

### `super()`
`super()` helps call parent behavior without hard-coding the parent class name.
- Makes code easier to maintain.
- Plays nicely with multiple inheritance.

### Method overriding
A subclass can override a method to change behavior.
- You can still call the parent method via `super().method()`.


In [ ]:
# Examples 13–15: inheritance + overriding + super

# Example 13: basic inheritance
class Animal:
    def speak(self):
        return "(silence)"

class Cat(Animal):
    def speak(self):
        return "meow"

print("Example 13:", Animal().speak(), Cat().speak())

# Example 14: super() to reuse parent init
class Employee:
    def __init__(self, name, base_salary):
        self.name = name
        self.base_salary = float(base_salary)

    def total_compensation(self):
        return self.base_salary

class Manager(Employee):
    def __init__(self, name, base_salary, bonus):
        super().__init__(name, base_salary)
        self.bonus = float(bonus)

    def total_compensation(self):
        return super().total_compensation() + self.bonus

m = Manager("Rita", 100_000, 20_000)
print("Example 14:", m.name, m.total_compensation())

# Example 15: isinstance checks inheritance hierarchy
print("Example 15:", isinstance(m, Manager), isinstance(m, Employee))


## 6) Polymorphism (duck typing)

### The idea
**Polymorphism** means “many forms”: different object types can be used through the same interface.

In Python, polymorphism often comes from **duck typing**:
> If it walks like a duck and quacks like a duck, treat it like a duck.

Instead of checking exact types, you often rely on whether an object provides the methods you need.

Why this matters:
- You can write generic code that works for lists, tuples, numpy arrays, custom containers, etc.
- Many ML libraries rely on consistent method names (`fit`, `predict`, `transform`).


In [ ]:
# Examples 16–17: polymorphism (duck typing)

# Example 16: same function, different object types
class Circle:
    def __init__(self, r):
        self.r = float(r)

    def area(self):
        return 3.14159 * self.r * self.r

class Rectangle:
    def __init__(self, w, h):
        self.w = float(w)
        self.h = float(h)

    def area(self):
        return self.w * self.h

def print_area(shape):
    # no type checks: we just assume it has .area()
    print("area=", shape.area())

print("Example 16:")
print_area(Circle(2))
print_area(Rectangle(3, 4))

# Example 17: a 'model-like' interface
class MeanModel:
    def fit(self, X):
        self.mean_ = sum(X) / len(X)
        return self

    def predict(self, X):
        return [self.mean_] * len(X)

model = MeanModel().fit([1, 2, 3, 10])
print("Example 17:", model.predict([0, 0, 0]))


## 7) Abstract Base Classes (ABCs): enforcing an interface

Duck typing is flexible, but sometimes you want a stronger guarantee that subclasses implement required methods.

### ABC basics
- You define an abstract base class using `abc.ABC`.
- Mark required methods with `@abstractmethod`.
- Python prevents direct instantiation of a class that still has unimplemented abstract methods.

This is useful for large codebases and team projects, where you want consistent behavior across implementations.


In [ ]:
# Examples 18–19: ABCs

# Example 18: define an interface with abc
from abc import ABC, abstractmethod

class Transformer(ABC):
    @abstractmethod
    def fit(self, X):
        ...

    @abstractmethod
    def transform(self, X):
        ...

class MinMaxScaler(Transformer):
    def fit(self, X):
        self.min_ = min(X)
        self.max_ = max(X)
        return self

    def transform(self, X):
        denom = (self.max_ - self.min_) or 1.0
        return [(x - self.min_) / denom for x in X]

scaler = MinMaxScaler().fit([2, 4, 10])
print("Example 18:", scaler.transform([2, 4, 10]))

# Example 19: abstract class can't be instantiated
try:
    Transformer()
except TypeError as e:
    print("Example 19: TypeError ->", e)


## 8) Composition vs inheritance

### Composition
**Composition** means building objects from other objects.
- “Has-a” relationship.
- Often more flexible than inheritance.

Example:
- A `ModelTrainer` **has a** `Model` and **has a** `Metric`.

### Why composition is often preferred
- Reduces tight coupling between classes
- Easier to swap components (strategy pattern)
- Avoids complex inheritance hierarchies


In [ ]:
# Examples 20–21: composition

# Example 20: composition with pluggable dependency
class Accuracy:
    def __call__(self, y_true, y_pred):
        correct = sum(int(a == b) for a, b in zip(y_true, y_pred))
        return correct / len(y_true)

class AlwaysZeroModel:
    def fit(self, X, y):
        return self

    def predict(self, X):
        return [0] * len(X)

class Trainer:
    def __init__(self, model, metric):
        self.model = model
        self.metric = metric

    def train_and_evaluate(self, X, y):
        self.model.fit(X, y)
        preds = self.model.predict(X)
        return self.metric(y, preds)

trainer = Trainer(AlwaysZeroModel(), Accuracy())
print("Example 20:", trainer.train_and_evaluate([1, 2, 3, 4], [0, 1, 0, 0]))

# Example 21: swap the component without changing Trainer
class AlwaysOneModel(AlwaysZeroModel):
    def predict(self, X):
        return [1] * len(X)

trainer2 = Trainer(AlwaysOneModel(), Accuracy())
print("Example 21:", trainer2.train_and_evaluate([1, 2, 3, 4], [0, 1, 0, 0]))


## 9) `dataclass`: less boilerplate for data containers

When a class is mostly used to store data (like a record), Python’s `dataclasses` module can auto-generate:
- `__init__`
- `__repr__`
- `__eq__` (equality)

This is great for configuration objects in ML experiments.


In [ ]:
# Examples 22–23: dataclasses

# Example 22: basic dataclass
from dataclasses import dataclass, field

@dataclass
class ExperimentConfig:
    lr: float
    batch_size: int
    epochs: int = 10

cfg = ExperimentConfig(lr=0.001, batch_size=32)
print("Example 22:", cfg)

# Example 23: default factory for mutable fields
@dataclass
class Dataset:
    name: str
    labels: list[str] = field(default_factory=list)

ds = Dataset("demo")
ds.labels.append("cat")
print("Example 23:", ds)


## 10) Operator overloading (dunder methods)

Python lets your classes behave like built-ins by implementing special methods (often called “dunder” methods):
- `__len__` for `len(obj)`
- `__add__` for `obj1 + obj2`
- `__getitem__` for indexing `obj[i]`

Used well, this improves readability.
Used poorly, it can surprise readers—so keep it intuitive.


In [ ]:
# Examples 24–25: operator overloading

# Example 24: __len__ + __getitem__ for a dataset-like container
class SimpleDataset:
    def __init__(self, items):
        self._items = list(items)

    def __len__(self):
        return len(self._items)

    def __getitem__(self, idx):
        return self._items[idx]

dset = SimpleDataset(["a", "b", "c"])
print("Example 24:", len(dset), dset[0], dset[-1])

# Example 25: __add__ to add vectors
class Vec:
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)

    def __add__(self, other):
        if not isinstance(other, Vec):
            return NotImplemented
        return Vec(self.x + other.x, self.y + other.y)

    def __repr__(self):
        return f"Vec({self.x}, {self.y})"

v1 = Vec(1, 2)
v2 = Vec(3, 4)
print("Example 25:", v1 + v2)


## 11) (Bonus) Context managers: `with` and `__enter__` / `__exit__`

### Why it matters
Context managers help manage resources safely:
- Files, database connections, locks
- Temporary state changes

Even if errors occur, cleanup still happens.

Below is a simple example that logs when a block starts/ends.


In [ ]:
# Example 26: context manager

class Timer:
    def __enter__(self):
        import time
        self._time = time
        self.start = time.time()
        return self

    def __exit__(self, exc_type, exc, tb):
        end = self._time.time()
        self.elapsed = end - self.start
        # return False means exceptions (if any) are not suppressed
        return False

with Timer() as t:
    total = sum(range(1_0000))
print("Example 26:", "elapsed_sec=", round(t.elapsed, 6), "total=", total)


## Quick practice (optional)
1. Create a `Student` class with `name`, `marks` (list), and a method `average()`.
2. Build a `Normalizer` ABC with `fit` and `transform`, and implement `ZScoreNormalizer`.
3. Use composition: `Pipeline` that holds a list of transformers and applies them in order.

---
You now have a solid OOP foundation for writing clean, reusable ML code.
